In [76]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image

# 加载预训练的Faster R-CNN模型
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# 定义函数来加载图像并将其转换为模型可以处理的格式
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = F.to_tensor(image)
    return image

# 定义函数来预测图像中的苹果数量
def count_apples(image_path, threshold=0.5):
    image = load_image(image_path)
    with torch.no_grad():
        prediction = model([image])
    
    # 过滤低概率结果
    pred_classes = [int(i) for i in list(prediction[0]['labels'])]
    pred_scores = list(prediction[0]['scores'])
    pred_count = sum(score > threshold and pred_class == 948  # 948是苹果的ImageNet类别索引
                     for score, pred_class in zip(pred_scores, pred_classes))
    
    return pred_count

# 使用模型
image_path = "../Attachment/Attachment 1-Augmented/augmented/83.jpg"  # 替换为您的图像文件路径
apple_count = count_apples(image_path)
print(f"Detected apples: {apple_count}")


d:\Download\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Download\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Download\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\lai

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import random_split
import os
import torch.nn.functional as F
from PIL import Image

In [18]:
class SimpleAppleDetector(nn.Module):
    def __init__(self):
        super(SimpleAppleDetector, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1) # assuming input image has 3 channels (RGB)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 22 * 33, 512) # assuming input image size is 224x224
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1) # output 1 value: number of apples

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 22 * 33) # flatten image input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = SimpleAppleDetector()
print(model)


SimpleAppleDetector(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=46464, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
)


In [11]:
image_count = 0
data_x, data_y = [], []

image_path = "../Attachment/Attachment 1-Augmented/augmented/"
for path in os.listdir(image_path):
    image_count += 1
    image = Image.open(image_path + path)
    image = list(np.array(image))
    data_x.append(image)
    
annonation_path = "../Attachment/Attachment 1-Annotation/"
for path in os.listdir(annonation_path):
    try:
        num_of_apples = 0
        f = open(annonation_path + path)
        data = json.load(f)
        for i in data['shapes']:
            num_of_apples += 1
        data_y.append(len)
    except:
        data_y.append(0)

data_x = np.array(data_x)
data_y = np.array(data_y)
print(data_x.shape)
print(data_y.shape)

(200, 185, 270, 3)
(200,)


In [19]:
num_epochs = 2
batch_size = 8
data_x = torch.tensor(np.array(data_x)).long()      
data_y = torch.tensor(np.array(data_y)).long()    
dataset = TensorDataset(data_x, data_y)
dataset_size = data_x.size(dim = 0)
train_size = int(dataset_size * 0.8)
test_size = int(dataset_size * 0.2)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
data_loader_train = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#data_loader_valid = DataLoader(dataset=dataset.valid, batch_size=batch_size, shuffle=False)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# 进行训练
for epoch in range(num_epochs):
    for images, labels in data_loader_train:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


RuntimeError: Input type (__int64) and bias type (float) should be the same